In [58]:
'''
Importing packages
Setting pandas to not display data in scientific notation.
'''
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.set_option('display.float_format', '{:.2f}'.format)

In [59]:
# Loading the csv file and getting a quick look at the data

df = pd.read_csv('../data/ultra_marathons.csv')
df.sample(5)

C:\Users\OMEN\AppData\Local\Temp\ipykernel_7804\1387840902.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/ultra_marathons.csv')


,Year of event,Event dates,Event name,Event distance/length,Event number of finishers,Athlete performance,Athlete club,Athlete country,Athlete year of birth,Athlete gender,Athlete age category,Athlete average speed,Athlete ID
3129772,2021,29.-31.10.2021,Ultra Trail Vipava Valley 160 Emperor (SLO),160km,45,36:05:33 h,*Amsterdam Binnenstad ...,USA,1989.00,M,M23,4.433,231701
3559679,2022,26.-27.11.2022,Bislett 24 Hour Indoor Challenge (NOR),24h,143,103.289 km,Bodø Bauta Løpeklubb,NOR,1991.00,F,W23,4.304,906364
6109927,2014,07.06.2014,Keufelskopf Ultra-Trail - offener Lauf (GER),85km,113,14:16:00 h,LT Hemsbach,GER,1955.00,M,M55,5.96,391584
4900005,2010,29.-30.05.2010,"100 km del Passatore, Firenze-Faenza (ITA)",100km,1027,14:20:40 h,Fattori Quarrata,ITA,1965.00,F,W45,6.971,90018
3665330,1997,30.-31.08.1997,Haliburton Forest 50 Mile Race (CAN),50mi,30,7:04:55 h,"*London, ON",CAN,1960.00,M,M35,11.36,71368


In [60]:
# Renaming all the columns using snake case

new_columns = {
    'Year of event': 'year_of_event',
    'Event dates': 'event_dates',
    'Event name': 'event_name',
    'Event distance/length': 'distance_or_length',
    'Event number of finishers': 'no_of_finishers',
    'Athlete performance': 'athlete_performance',
    'Athlete club': 'athlete_club',
    'Athlete country': 'athlete_country',
    'Athlete year of birth': 'birth_year',
    'Athlete gender': 'gender',
    'Athlete age category': 'age_category',
    'Athlete average speed': 'average_speed',
    'Athlete ID': 'athlete_id'
}
df.rename(columns = new_columns, inplace = True)
df.head()

,year_of_event,event_dates,event_name,distance_or_length,no_of_finishers,athlete_performance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id
0,2018,06.01.2018,Selva Costera (CHI),50km,22,4:51:39 h,Tnfrc,CHI,1978.00,M,M35,10.286,0
1,2018,06.01.2018,Selva Costera (CHI),50km,22,5:15:45 h,Roberto Echeverría,CHI,1981.00,M,M35,9.501,1
2,2018,06.01.2018,Selva Costera (CHI),50km,22,5:16:44 h,Puro Trail Osorno,CHI,1987.00,M,M23,9.472,2
3,2018,06.01.2018,Selva Costera (CHI),50km,22,5:34:13 h,Columbia,ARG,1976.00,M,M40,8.976,3
4,2018,06.01.2018,Selva Costera (CHI),50km,22,5:54:14 h,Baguales Trail,CHI,1992.00,M,M23,8.469,4


In [61]:
# Converting the Athlete year of birth column into integers when not null

df['birth_year'] = df['birth_year'][df['birth_year'].notnull()].astype(int)
df.head()

,year_of_event,event_dates,event_name,distance_or_length,no_of_finishers,athlete_performance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id
0,2018,06.01.2018,Selva Costera (CHI),50km,22,4:51:39 h,Tnfrc,CHI,1978.00,M,M35,10.286,0
1,2018,06.01.2018,Selva Costera (CHI),50km,22,5:15:45 h,Roberto Echeverría,CHI,1981.00,M,M35,9.501,1
2,2018,06.01.2018,Selva Costera (CHI),50km,22,5:16:44 h,Puro Trail Osorno,CHI,1987.00,M,M23,9.472,2
3,2018,06.01.2018,Selva Costera (CHI),50km,22,5:34:13 h,Columbia,ARG,1976.00,M,M40,8.976,3
4,2018,06.01.2018,Selva Costera (CHI),50km,22,5:54:14 h,Baguales Trail,CHI,1992.00,M,M23,8.469,4


In [62]:
# Checking null value counts per column
df.isnull().sum()

year_of_event                0
event_dates                  0
event_name                   0
distance_or_length        1053
no_of_finishers              0
athlete_performance          2
athlete_club           2826524
athlete_country              3
birth_year              588161
gender                       7
age_category            584938
average_speed              224
athlete_id                   0
dtype: int64

In [63]:
# Checking Athlete gender column value counts, there are some bad values here, that are labeled X.

df['gender'].value_counts()

gender
M    6035358
F    1425784
X         46
Name: count, dtype: int64

In [64]:
# Finding all the X gender values, 

x_gender = df[df['gender'] == 'X']['athlete_id'].unique()
df[df['athlete_id'].isin(list(x_gender))].sort_values(by = 'gender', ascending = False).head(5)

,year_of_event,event_dates,event_name,distance_or_length,no_of_finishers,athlete_performance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id
73627,2018,21.01.2018,Zhenxibao (Cinsbu) 54 km Ultramarathon (TPE),54km,926,8:48:43 h,NaN,TPE,1988.00,X,NaN,6.13,63250
3509473,2022,24.09.2022,Ochil Ultra 50mi (GBR),50mi,33,10:39:20 h,NaN,GBR,1980.00,X,NaN,7.552,1023739
3054941,2021,25.09.2021,Ochil Ultra 50mi (GBR),50mi,51,13:02:18 h,NaN,GBR,1971.00,X,NaN,6.172,420655
3247275,2022,12.03.2022,Marin Ultra Challenge 50 km (USA),50km,236,6:16:36 h,"*Sacramento, CA",USA,1975.00,X,NaN,7.966,201998
3365115,2022,18.06.2022,Rachel Carson Trail Challenge (USA),34mi,486,14:40:02 h,"*Monrovia, CA",USA,1975.00,X,NaN,3.731,993606


In [65]:
# Of the improper values there are two athlete ids that have athlete category values that tell me what their gender actually is

df['gender'][df['athlete_id'].isin([420655, 609724])] = 'M'
df[df['gender'] == 'X'].head()

C:\Users\OMEN\AppData\Local\Temp\ipykernel_7804\2322290404.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gender'][df['athlete_id'].isin([420655, 609724])] = 'M'


,year_of_event,event_dates,event_name,distance_or_length,no_of_finishers,athlete_performance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id
73627,2018,21.01.2018,Zhenxibao (Cinsbu) 54 km Ultramarathon (TPE),54km,926,8:48:43 h,NaN,TPE,1988.00,X,NaN,6.13,63250
151581,2018,08.04.2018,Cape Cod 50K Trail Race (USA),50km,50,6:26:26 h,"*Plymouth, MA",USA,1999.00,X,NaN,7.763,120517
291422,2018,19.05.2018,Silver State 50 Mile (USA),50mi,79,12:22:41 h,"*San Francisco, CA",USA,1975.00,X,NaN,6.50,201998
596645,2018,08.12.2018,Hunter S. Thompson Fear & Loathing 50 Km (USA),50km,47,5:48:00 h,"*San Francisco, CA",USA,1975.00,X,NaN,8.621,201998
617444,2018,24.11.2018,Quad Dipsea Trail Run (USA),28mi,279,5:51:40 h,"*San Francisco, CA",USA,1975.00,X,NaN,7.688,201998


In [66]:
# Checking out all the value counts for each age category, there are some improper values
# that start with f, mu or wu instead of m or w.

df['age_category'].value_counts().sort_index()

age_category
F35           1
M20       18836
M23     1228209
M30       23493
M35      989217
M40     1054195
M45      908337
M50      626637
M55      351116
M60      168131
M65       66603
M70       21698
M75        5502
M80        1225
M85         164
M90          19
M95          28
MU20       1355
MU23     100183
W20        2890
W23      314473
W30        4385
W35      245020
W40      264967
W45      221291
W50      135437
W55       64656
W60       25664
W65        8810
W70        2850
W75         873
W80         123
W85           8
W90           2
W95           3
WU20        316
WU23      19540
Name: count, dtype: int64

In [67]:
# Changing the one f35 value to w35 which conforms to the standards of the majority of values in age category.

df.loc[df['age_category'] == 'F35', 'age_category'] = 'W35'

In [68]:
'''
Creating a mask and function to be used to fix the values in age category.
The mask checks for all values in the mu*, wu* or for any empty values as well.
The function basically takes the year of the event - birth year to get the age of the
athlete at the time and then returns it. 
'''

old_age_cat_mask = (df['age_category'].isnull()) | (df['age_category'].isin(['MU23', 'MU20', 'WU20', 'WU23']))

def change_age_cats(row):
    if row['gender'] == 'F':
        gender = 'W'
    elif row['gender'] == 'M':
        gender = 'M'
    else:
        gender = 'X'
    try:
        birth_year = int(row['birth_year'])
        event_year = int(row['year_of_event'])
    except:
        return gender + '0'
    age = event_year - birth_year
    age_group = int(age) if age % 23 == 0 else int(age / 5) * 5
    return gender + str(abs(age_group))


df.loc[old_age_cat_mask, 'age_category'] = df[old_age_cat_mask].apply(change_age_cats, axis=1)

In [69]:
# Making a check to ensure that there are no improper values, and that everything was categorized properly

print(df['age_category'].isnull().sum())
df['age_category'].value_counts().sort_index()

0


age_category
M0       470493
M10        1527
M110          3
M130          2
M135          1
M15       18706
M20       75299
M23     1252725
M25           8
M30       23503
M35      989235
M40     1054207
M45      908348
M46           2
M5          155
M50      626647
M55      351124
M60      168135
M65       66603
M70       21698
M75        5502
M80        1225
M825          1
M85         164
M90          19
M95          28
W0       114465
W10         591
W115          1
W15        3304
W20       12956
W23      320303
W25           1
W30        4393
W35      245023
W40      264970
W45      221295
W5           58
W50      135438
W55       64657
W60       25664
W65        8810
W70        2850
W75         873
W80         123
W85           8
W90           2
W95           3
X0            5
X15           2
X23           1
X25           2
X30           4
X35           7
X40          20
X45           4
X46           2
Name: count, dtype: int64

In [70]:
# Checking to see if there are any outliers or improper values, and there is only one improper value,
# a birth year just after the year 1100. The data here is for races only since the late 1700s.
    
df['birth_year'].describe()

count   6873034.00
mean       1969.76
std          13.13
min        1193.00
25%        1962.00
50%        1971.00
75%        1979.00
max        2021.00
Name: birth_year, dtype: float64

In [71]:
# Taking a closer look at the outlier values, I would assume that this person was born in 1993, but I don't want to change the

df[df['birth_year'] < (2023 - 90)].sort_values(by = 'birth_year', ascending = True)

,year_of_event,event_dates,event_name,distance_or_length,no_of_finishers,athlete_performance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id
2582496,2020,19.06.2020,Dalat Ultra Trail - 45km (VIE),45km,224,8:49:50 h,Adidasrunnerssaigon,VIE,1193.00,M,M825,5.096,819254
6824337,1807,12.-13.10.1807,24 h Newmarket (GBR),24h,2,64.374 km,*Lancashire,GBR,1786.00,M,M20,2682.00,1520516
6824339,1837,30.01.1837,London to Brighton Race (GBR),85km,1,8:37:00 h,NaN,GBR,1791.00,M,M45,9865.00,340384
6824825,1892,02.-03.09.1892,100 km Geneve-Preverenges-Geneve (SUI),100km,10,22:00:00 h,*Anières,SUI,1816.00,M,M75,4545.00,1520678
6825034,1892,05.-15.06.1892,Paris-Belfort (FRA),496km,29,9d 05:57:00 h,*Douai Le Doyen,FRA,1817.00,M,M70,2235.00,1520872
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6985148,1982,18.04.1982,American River 50 Mile Endurance Run (USA),50mi,324,9:37:09 h,NaN,USA,1932.00,M,M45,8365.00,1571286
6985195,1982,18.04.1982,American River 50 Mile Endurance Run (USA),50mi,324,10:06:25 h,NaN,USA,1932.00,M,M45,7962.00,1564063
6985220,1982,18.04.1982,American River 50 Mile Endurance Run (USA),50mi,324,10:24:41 h,NaN,USA,1932.00,M,M45,7729.00,1574678
6984764,1982,24.04.1982,"100 km du Périgord Noir, Belves (FRA)",100km,393,15:31:00 h,*Volvic,FRA,1932.00,M,M45,6445.00,1570738


In [72]:
# Looking to see if there is any more data for the one incorrect birth year.
# There is no more data on this particular individual, so his birth year will have to be nulled

df[df['athlete_id'] == 819254]

,year_of_event,event_dates,event_name,distance_or_length,no_of_finishers,athlete_performance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id
2582496,2020,19.06.2020,Dalat Ultra Trail - 45km (VIE),45km,224,8:49:50 h,Adidasrunnerssaigon,VIE,1193.00,M,M825,5.096,819254


In [73]:
# Setting the birth year for the outlier to np.nan

df.loc[df['athlete_id'] == 819254, 'birth_year'] = np.nan

In [74]:
df['year_of_event'].describe()

count   7461195.00
mean       2011.98
std          10.01
min        1798.00
25%        2010.00
50%        2015.00
75%        2018.00
max        2022.00
Name: year_of_event, dtype: float64

In [75]:
df['athlete_club'].fillna('Unknown', inplace = True)
df['athlete_club'].isnull().sum()

0

In [76]:
'''
Checking to see what different kind of values are in the distance or length column
I will have to convert these values to numerics and separate them into a distance and
a time column, as well as create a metric column to identify whether the race uses miles,
kilometers, or hours.
'''

df['distance_or_length'].value_counts()

distance_or_length
50km              1522609
100km              951742
50mi               352181
56km               333253
87km               212062
                   ...   
61.15km                 1
27:30h                  1
178km/2Etappen          1
740km                   1
158.5mi                 1
Name: count, Length: 2159, dtype: int64

In [77]:
# Checking on the types to see if they conform to the standards that I want, they do
# not but this will be fixed.

df.dtypes

year_of_event            int64
event_dates             object
event_name              object
distance_or_length      object
no_of_finishers          int64
athlete_performance     object
athlete_club            object
athlete_country         object
birth_year             float64
gender                  object
age_category            object
average_speed           object
athlete_id               int64
dtype: object

In [78]:
'''
Checking to see which values in the average speed column are in float format and which are not.
Upon looking at the average speed data, I can see that it isn't clear what format or measurement
the values are in, it is very inconsistent, and I do not think that I will use it.
'''

def can_convert_to_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

df['can_convert'] = df['average_speed'].apply(can_convert_to_float)

# DataFrame with convertible values
convertible_df = df[df['can_convert']]

# DataFrame with non-convertible values
non_convertible_df = df[~df['can_convert']]

print("Convertible to Float:")
print(convertible_df['average_speed'].value_counts())

print("\nNot Convertible to Float:")
print(non_convertible_df['average_speed'].value_counts())
df.drop('can_convert', axis = 1, inplace = True)


Convertible to Float:
average_speed
7         3210
8         3098
4.167     2864
0         2857
10        2421
          ... 
15.654       1
15.547       1
15.426       1
29.185       1
4128.0       1
Name: count, Length: 45918, dtype: int64

Not Convertible to Float:
average_speed
07:00:00    977
12:00:00    790
08:00:00    774
10:00:00    742
00:00:00    691
           ... 
08:31:30      1
08:28:02      1
07:54:04      1
07:47:04      1
09:48:55      1
Name: count, Length: 1326, dtype: int64


In [79]:
# Using regex to create a new column race metric for determining whether the race is in km, mi or hours.
# Also converting distance or length into a numerical valued column and getting rid of alphabetic suffixes.

pattern = r'(\d+)([a-zA-Z]+)'

df[['distance_or_length', 'race_metric']] = df['distance_or_length'].str.extract(pattern)

df['distance_or_length'] = pd.to_numeric(df['distance_or_length'], errors='coerce')
df.head()

,year_of_event,event_dates,event_name,distance_or_length,no_of_finishers,athlete_performance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id,race_metric
0,2018,06.01.2018,Selva Costera (CHI),50.00,22,4:51:39 h,Tnfrc,CHI,1978.00,M,M35,10.286,0,km
1,2018,06.01.2018,Selva Costera (CHI),50.00,22,5:15:45 h,Roberto Echeverría,CHI,1981.00,M,M35,9.501,1,km
2,2018,06.01.2018,Selva Costera (CHI),50.00,22,5:16:44 h,Puro Trail Osorno,CHI,1987.00,M,M23,9.472,2,km
3,2018,06.01.2018,Selva Costera (CHI),50.00,22,5:34:13 h,Columbia,ARG,1976.00,M,M40,8.976,3,km
4,2018,06.01.2018,Selva Costera (CHI),50.00,22,5:54:14 h,Baguales Trail,CHI,1992.00,M,M23,8.469,4,km


In [80]:
# Checking to see the differing values in race metric, there are many versions of 
# the same thing here which will all need to be changed.

df['race_metric'].value_counts()

race_metric
km         6154371
mi          775930
h           514457
d            12190
Km            1931
Miles          264
miles          234
k              159
Mile            64
mile            58
K               52
m               40
Etappen          5
x                4
Name: count, dtype: int64

In [81]:
# There are some two strange race metric values, x and etappen, which means that it is a stage race.
# A stage race is like multiple races, where each one finishes and after or a day later they will have the second stage of the race.
# The data is clear that the format is in hours for the Notchview Ultra Quadzilla ultra

df[df['race_metric'] == 'x']

,year_of_event,event_dates,event_name,distance_or_length,no_of_finishers,athlete_performance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id,race_metric
2103612,2019,04.-07.07.2019,Notchview Ultra Quadzilla Ultra (USA),4.00,4,38:31:27 h,"*New Hartford, CT",USA,1977.00,M,M40,0,101669,x
2103613,2019,04.-07.07.2019,Notchview Ultra Quadzilla Ultra (USA),4.00,4,47:11:01 h,"*New Hartford, NY",USA,1996.00,F,W23,0,129191,x
2103614,2019,04.-07.07.2019,Notchview Ultra Quadzilla Ultra (USA),4.00,4,2d 12:44:40 h,"*Rochester, NY",USA,1964.00,M,M50,0,78349,x
2103615,2019,04.-07.07.2019,Notchview Ultra Quadzilla Ultra (USA),4.00,4,2d 20:49:30 h,"*Worcester, MA",USA,1991.00,M,M23,0,181853,x


In [82]:
# Consolidating the data into proper values

metric_conversion = {
    'miles': 'mi',
    'Miles': 'mi',
    'mile': 'mi',
    'Mile': 'mi',
    'm': 'mi',
    'k': 'km',
    'K': 'km',
    'Km': 'km',
    'Etappen': 'hours',
    'h': 'hours',
    'd': 'days',
    'x': 'hours'
}

df['race_metric'] = df['race_metric'].replace(metric_conversion)
df['race_metric'].value_counts()

race_metric
km       6156513
mi        776590
hours     514466
days       12190
Name: count, dtype: int64

In [83]:
# Filling in any faulty birth year values with 0

df['birth_year'] = df['birth_year'].fillna(0).astype(int)

In [84]:
# Checking to see if there are any unusual values in the country category, and it seems like everything is fine

print(df['athlete_country'].value_counts())
df[(df['athlete_country'].str.len() > 3)  | (df['athlete_country'].str.len() < 3)]

athlete_country
USA    1389960
FRA    1170884
RSA     877630
JPN     603132
GER     442056
        ...   
AHO          1
SLE          1
BAR          1
TGA          1
ACT          1
Name: count, Length: 208, dtype: int64


,year_of_event,event_dates,event_name,distance_or_length,no_of_finishers,athlete_performance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id,race_metric


In [85]:
'''
The following 3 cells and this one took more time to complete than anything else.
At first I created a for loop that got everything done in one cell, but there was a very strange issue
where it would try to convert all the dates to date time and the date '18.03.2018' kept throwing an
error saying that the day was out of range for the month, which is completely incorrect.
So after several hours I decided to just compartmentalize everything into separate cells, and everything
worked almost immediately.
'''

df['event_dates'] = df['event_dates'].str.replace('00', '01')
mask1 = df['event_dates'].str.len() == 10
single_dates = pd.to_datetime(df.loc[mask1, 'event_dates'], format = '%d.%m.%Y')
df.loc[mask1, 'start_date'] = df.loc[mask1, 'end_date'] = single_dates

In [86]:
mask2 = df['event_dates'].str.len() == 14
date_ranges1 = df.loc[mask2, 'event_dates'].str.split('-', expand = True)
end = pd.to_datetime(date_ranges1[1], format='%d.%m.%Y', errors = 'coerce')
start = pd.to_datetime(date_ranges1[0] + date_ranges1[1].str.slice(start = 3), format = '%d.%m.%Y')
df.loc[mask2, 'start_date'] = start
df.loc[mask2, 'end_date'] = end

In [87]:
mask3 = df['event_dates'].str.len() == 17
date_ranges2 = df.loc[mask3, 'event_dates'].str.split('-', expand = True)
end = pd.to_datetime(date_ranges2[1], format='%d.%m.%Y', errors = 'coerce')
start = pd.to_datetime(date_ranges2[0] + date_ranges2[1].str.slice(start = 6), format = '%d.%m.%Y')
df.loc[mask3, 'start_date'] = start
df.loc[mask3, 'end_date'] = end

In [88]:
mask4 = df['event_dates'].str.len() == 21
date_ranges3 = df.loc[mask4, 'event_dates'].str.split('-', expand = True)
start = pd.to_datetime(date_ranges3[0], format = '%d.%m.%Y')
end = pd.to_datetime(date_ranges3[1], format='%d.%m.%Y', errors = 'coerce')
df.loc[mask4, 'start_date'] = start
df.loc[mask4, 'end_date'] = end

In [89]:
# Checking the dates, ensuring accurate and proper data creation

df.sample(5)

,year_of_event,event_dates,event_name,distance_or_length,no_of_finishers,athlete_performance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id,race_metric,start_date,end_date
6911333,1979,08.-09.06.1979,100 km Lauf Biel (SUI),100.00,2943,13:05:00 h,*Olten,BEL,1953,M,M23,7643.00,1537421,km,1979-06-08,1979-06-09
3559529,2022,26.-27.11.2022,Kyoto Ultra Walking 110km (JPN),110.00,217,26:57:10 h,Unknown,JPN,0,M,M0,4.081,1034347,km,2022-11-26,2022-11-27
402524,2018,17.06.2018,Self-Transcendence 50km Paris (FRA),50.00,21,4:38:38 h,US Roissy En Brie,FRA,1971,F,W45,10.767,102418,km,2018-06-17,2018-06-17
673382,2016,11.-12.03.2016,TransLantau 100 Ultra Trail (HKG),100.00,514,22:10:42 h,Sunday Runner,MAS,1964,M,M50,4.509,285876,km,2016-03-11,2016-03-12
5241850,2011,02.10.2011,Trail des Glieres (FRA),5.00,132,5:29:07 h,Unknown,FRA,1973,M,M35,9.94,1230001,km,2011-10-02,2011-10-02


In [90]:
# Dropping the now unnecessary event dates column

df.drop(['event_dates'], axis = 1, inplace = True)

In [91]:
# Looking at the data once more for null values

df.isnull().sum()

year_of_event             0
event_name                0
distance_or_length     1436
no_of_finishers           0
athlete_performance       2
athlete_club              0
athlete_country           3
birth_year                0
gender                    7
age_category              0
average_speed           224
athlete_id                0
race_metric            1436
start_date                0
end_date                 19
dtype: int64

In [92]:
df.sample(5)

,year_of_event,event_name,distance_or_length,no_of_finishers,athlete_performance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id,race_metric,start_date,end_date
1926434,2019,Trail de l'Aber Wrach - Extrême Trail (FRA),56.00,535,7:22:08 h,Unknown,FRA,1983,M,M35,7.60,141819,km,2019-04-13,2019-04-13
2207980,2019,Els Bastions 50 kms Marathon (ESP),50.00,342,10:11:02 h,Unknown,ESP,1964,M,M50,4.91,734883,km,2019-06-01,2019-06-01
158406,2018,South Downs Way 50 Mile Race (GBR),50.00,354,10:03:57 h,Unknown,GBR,1979,M,M35,7.994,124680,mi,2018-04-07,2018-04-07
4717107,2009,De Zestig van Texel (NED),60.00,241,6:19:19 h,Krekenlopers St. Laureins,BEL,1949,M,M55,9.491,64725,km,2019-04-13,2019-04-13
5126936,2011,Lavaredo Ultra Trail (ITA),90.00,420,21:31:48 h,CAF IDF,FRA,1962,M,M45,4.18,1252449,km,2011-07-02,2011-07-02


In [93]:
'''
Creating a new column called fastest time which takes the data from athlete performance
and then if it is in the format of hours puts it in that column, leaving all the data
in the form of kilometers or miles in the old column, and then changing the name to total
distance.
'''
df['athlete_performance'] = df['athlete_performance'].str.replace(' h', '')
df['fastest_time'] = df['athlete_performance']
df.loc[df['fastest_time'].str.contains('k', na = False), 'fastest_time'] = '0'
df.loc[df['athlete_performance'].str.contains(':', na = False), 'athlete_performance'] = '0'
df.rename(columns = {'athlete_performance': 'total_distance'}, inplace = True)
df.sample(5)

,year_of_event,event_name,distance_or_length,no_of_finishers,total_distance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id,race_metric,start_date,end_date,fastest_time
4026394,2002,Holiday Lake 50K++ (USA),50.00,167,0,Unknown,USA,1957,M,M40,7.05,1094637,km,2012-02-09,2012-02-09,7:05:34
5870051,2013,Chicago Lakefront Fall 50K (USA),50.00,178,0,"*Saint Charles, IL",USA,1974,F,W35,8.794,1415567,km,2013-11-02,2013-11-02,5:41:08
6002965,2014,Monteregio Ultra-Trail 47 km (ITA),47.00,126,0,Podistica Mezzanese,ITA,1975,M,M35,7.684,89879,km,2014-04-06,2014-04-06,6:06:59
7421161,1995,La Grande Course des Templiers (FRA),65.00,367,0,Unknown,FRA,0,M,M0,8535.00,1638646,km,1995-10-28,1995-10-28,7:36:55
1460055,2017,OSJ Ontake 100km Ultra Trail (JPN),100.00,866,0,*Shizuoka Pref.,JPN,1973,M,M40,6.965,97018,km,2017-07-15,2017-07-15,14:21:28


In [94]:
# Not sure why there are a bunch of asterisks in the athlete club column, but I am getting rid of them.
# Maybe it is for when an athlete has their home city as their club, rather than having an actual running club.

df['athlete_club'] = df['athlete_club'].str.replace('*', '')
df.sample(10)

,year_of_event,event_name,distance_or_length,no_of_finishers,total_distance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id,race_metric,start_date,end_date,fastest_time
6252918,2014,LangsteNachtLoop 24 uurs - 12h Split (NED),12.00,19,98.407 km,Unknown,NED,1969,M,M45,8.201,62545,hours,2014-12-20,2014-12-20,0
3285150,2022,Kerry Ultra Marathon 50K (IRL),50.00,56,0,Unknown,IRL,0,M,M0,8.568,976587,km,2022-05-14,2022-05-14,5:50:08
2192570,2019,Tenerife Blue Trail 67 km (ESP),67.00,317,0,Club Veredas A Tauro,ESP,1976,M,M40,5.946,731467,km,2019-06-08,2019-06-08,11:16:06
4000028,2002,GutsMuths-Rennsteiglauf - DUV-DM im Cross- und...,5.00,1383,0,Das Coyoten Team,GER,1971,M,M30,8.84,606071,km,2012-05-25,2012-05-25,8:25:46
5193525,2011,New Years One Day 6 Hour Run (USA),6.00,13,49.517 km,Unknown,USA,1961,M,M50,8.25,503093,hours,2011-12-31,2012-01-01,0
940886,2016,Ultra Brecon 50 - Summer (GBR),42.00,34,0,Unknown,GBR,1974,M,M40,7.215,418716,mi,2016-05-14,2016-05-14,9:22:06
3188613,2022,Zane Grey 100km Endurance Run (USA),100.00,57,0,"Scottsdale, AZ",USA,1979,M,M40,6.483,557640,km,2022-04-23,2022-04-23,15:25:27
69085,2018,Hong Kong 100 Ultra Trail Race (HKG),100.00,1519,0,Unknown,CHN,1982,F,W35,5.26,60019,km,2018-01-27,2018-01-28,19:00:33
3054737,2021,The Big Schloss 50K (USA),50.00,47,0,Unknown,USA,1970,M,M50,6.024,182970,km,2021-09-25,2021-09-25,8:18:00
3508992,2022,Leinetal-Ultra (GER),65.00,30,0,Team Vegan Straight Ed...,GER,1985,M,M35,6.555,833244,km,2022-09-24,2022-09-24,9:55:00


In [95]:
# Quick data type check, total distance needs to be converted to float

df.dtypes

year_of_event                  int64
event_name                    object
distance_or_length           float64
no_of_finishers                int64
total_distance                object
athlete_club                  object
athlete_country               object
birth_year                     int32
gender                        object
age_category                  object
average_speed                 object
athlete_id                     int64
race_metric                   object
start_date            datetime64[ns]
end_date              datetime64[ns]
fastest_time                  object
dtype: object

In [96]:
# Converting datatype for the total distance column to float

df['total_distance'] = df['total_distance'].str.replace(' km', '').astype(float)
df.sample(5)

,year_of_event,event_name,distance_or_length,no_of_finishers,total_distance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id,race_metric,start_date,end_date,fastest_time
5135272,2011,Western States 100 Mile Endurance Run (USA),100.00,310,0.00,"Mount Aukum, CA",USA,1950,M,M60,5.776,35588,mi,2011-06-25,2011-06-26,27:51:37
4206587,2005,La SainteLyon 69 km (FRA),69.00,1604,0.00,Crv Lyon Triathlon,FRA,1977,M,M23,10.02,1177398,km,2015-12-04,2015-12-04,6:53:21
2100243,2019,Bergmarathon Rund um den Traunsee (AUT),5.00,276,0.00,LC Weichstetten,AUT,1993,M,M23,5.375,711258,km,2019-07-06,2019-07-06,11:59:57
483190,2018,Beacons 50 (GBR),50.00,20,0.00,Running Forever RC,GBR,1964,M,M50,6.382,241270,mi,2018-08-17,2018-08-17,12:36:27
6154579,2014,Costa Esmeralda Ultra Trail 100km (BRA),100.00,84,0.00,Rio De Janeiro,BRA,1965,M,M45,4.57,1449150,km,2014-05-17,2014-05-17,21:51:40


In [97]:
# I noticed some strange values for athlete club, it seems that some rows have the data that should be in fastest time, in athlete club.

df[(df['average_speed'].isnull()) & (
    df['athlete_club'].str.contains('Performance'))].head()

,year_of_event,event_name,distance_or_length,no_of_finishers,total_distance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id,race_metric,start_date,end_date,fastest_time
404474,2018,Self-Transcendence 3100 Mile Race (USA),3100.00,6,0.00,Performance: 1072:03:53 h,RUS,1966,M,M50,NaN,262953,mi,2018-06-17,2018-08-07,0:00:00
404475,2018,Self-Transcendence 3100 Mile Race (USA),3100.00,6,0.00,Performance: 1107:24:48 h,ISR,1972,M,M45,NaN,262954,mi,2018-06-17,2018-08-07,0:00:00
404476,2018,Self-Transcendence 3100 Mile Race (USA),3100.00,6,0.00,Performance: 1207:34:46 h,AUT,1967,M,M50,NaN,206774,mi,2018-06-17,2018-08-07,0:00:00
404477,2018,Self-Transcendence 3100 Mile Race (USA),3100.00,6,0.00,Performance: 1236:47:37 h,AUT,1959,F,W55,NaN,82231,mi,2018-06-17,2018-08-07,0:00:00
404478,2018,Self-Transcendence 3100 Mile Race (USA),3100.00,6,0.00,Performance: 1240:46:38 h,BUL,1980,M,M35,NaN,262955,mi,2018-06-17,2018-08-07,0:00:00


In [98]:
# The purpose of this code is simply to take the time data from the incorrectly
# input data into the correct fastest time column from the athlete club column

wrong_athlete_club_vals = (df['average_speed'].isnull()) & (df['athlete_club'].str.contains('Performance'))
fix_athlete_vals = lambda row: pd.Series(['Unknown', row['athlete_club'].split(' ')[1]])

df.loc[wrong_athlete_club_vals, ['athlete_club', 'fastest_time']] = df[wrong_athlete_club_vals].apply(fix_athlete_vals, axis = 1)
df.sort_values(by = 'fastest_time', ascending = False)

,year_of_event,event_name,distance_or_length,no_of_finishers,total_distance,athlete_club,athlete_country,birth_year,gender,age_category,average_speed,athlete_id,race_metric,start_date,end_date,fastest_time
3828751,2000,Sri Chinmoy Ultra Trio 700 Mile Race (USA),700.00,12,0.00,"Jamaica, NY",USA,1952,M,M45,4.70,219882,mi,2010-09-10,2010-09-23,9d 23:51:20
2463668,2019,"Across the Years, 10 days - 1000km Split (USA)",1000.00,5,0.00,"Seattle, WA",USA,1975,F,W40,4.174,140909,km,2019-12-28,2020-01-07,9d 23:33:16
2463669,2019,"Across the Years, 10 days - 1000km Split (USA)",1000.00,5,0.00,"Seattle, WA",USA,1971,F,W45,4.174,11736,km,2019-12-28,2020-01-07,9d 23:33:16
2937253,2021,The Last Annual Heart of the South Road Race (...,326.00,46,0.00,"Blacksburg, VA",USA,1977,M,M40,2.191,1202,mi,2021-06-17,2021-06-27,9d 23:25:57
2937252,2021,The Last Annual Heart of the South Road Race (...,326.00,46,0.00,"Newland, NC",USA,1982,F,W35,2.191,901860,mi,2021-06-17,2021-06-27,9d 23:25:26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6633523,2015,Self-Transcendence 3100 Mile Race (USA),3100.00,8,0.00,NaN,CZE,1978,M,M35,NaN,441836,mi,2015-06-14,2015-08-04,NaN
6633524,2015,Self-Transcendence 3100 Mile Race (USA),3100.00,8,0.00,NaN,AUT,1959,F,W55,NaN,82231,mi,2015-06-14,2015-08-04,NaN
6633525,2015,Self-Transcendence 3100 Mile Race (USA),3100.00,8,0.00,NaN,UKR,1970,M,M40,NaN,441838,mi,2015-06-14,2015-08-04,NaN
6633526,2015,Self-Transcendence 3100 Mile Race (USA),3100.00,8,0.00,NaN,IRL,1980,M,M35,NaN,595711,mi,2015-06-14,2015-08-04,NaN


In [99]:
# Putting unknown for athletes with no club.

df['athlete_club'] = df['athlete_club'].fillna('Unknown')

In [100]:
df.isnull().sum()

year_of_event            0
event_name               0
distance_or_length    1436
no_of_finishers          0
total_distance           2
athlete_club             0
athlete_country          3
birth_year               0
gender                   7
age_category             0
average_speed          224
athlete_id               0
race_metric           1436
start_date               0
end_date                19
fastest_time           189
dtype: int64

In [101]:
# There are still 40 values that are improperly labeled in gender, but I think I will go on without them.
df['gender'].value_counts()

gender
M    6035364
F    1425784
X         40
Name: count, dtype: int64

In [102]:
# All done, creating two csv files, one for men the other for women

females = df[df['gender'] == 'F']
males = df[df['gender'] == 'M']
females.to_csv('../data/female_ultra_runners.csv')
males.to_csv('../data/male_ultra_runners.csv')

In [103]:
df.to_csv('../data/ultra_marathons_data_cleaned.csv', index = False)